**7.RNN.ipynb**

In [1]:
!wget https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Train.csv
!wget https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Test.csv

--2023-11-03 06:48:07--  https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Train.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/111-2/Google_Stock_Price_Train.csv [following]
--2023-11-03 06:48:07--  https://raw.githubusercontent.com/shhuangmust/AI/111-2/Google_Stock_Price_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63488 (62K) [text/plain]
Saving to: ‘Google_Stock_Price_Train.csv’

Google_Stock_Price_ 100%[===================>]  62.00K  --.-KB/s    in 0.02s   

2023-11-03 06:48:07 (2.50 MB/s) - ‘Google_Stock_Price_Train.csv’ saved [63488/63488]

--

In [2]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt  # for 畫圖用
import pandas as pd


# 匯入訓練資料集
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')  # 讀取訓練集
training_set = dataset_train.iloc[:, 1:2].values  # 取「Open」欄位值

# # 特徵縮放
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []   #預測點的前 60 天的資料
y_train = []   #預測點
for i in range(60, 1258):  # 1258 是訓練集總數
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)  # 轉成numpy array的格式，以利輸入 RNN

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [3]:
# # 導入Keras庫和模組
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# 初始化RNN模型
regressor = Sequential()
# 加入第一個LSTM層和一些Dropout正規化
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# 二
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# 三
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# 四
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
# 新增輸出層
regressor.add(Dense(units = 1))
# 編譯
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# 進行訓練
regressor.fit(X_train, y_train, epochs = 65, batch_size = 32)

Epoch 1/65
38/38 [==============================] - 13s 111ms/step - loss: 0.0359
Epoch 2/65
38/38 [==============================] - 4s 112ms/step - loss: 0.0063
Epoch 3/65
38/38 [==============================] - 6s 148ms/step - loss: 0.0053
Epoch 4/65
38/38 [==============================] - 4s 113ms/step - loss: 0.0056
Epoch 5/65
38/38 [==============================] - 7s 200ms/step - loss: 0.0048
Epoch 6/65
38/38 [==============================] - 5s 125ms/step - loss: 0.0048
Epoch 7/65
38/38 [==============================] - 4s 114ms/step - loss: 0.0044
Epoch 8/65
38/38 [==============================] - 8s 225ms/step - loss: 0.0046
Epoch 9/65
38/38 [==============================] - 4s 113ms/step - loss: 0.0046
Epoch 10/65
38/38 [==============================] - 5s 145ms/step - loss: 0.0050
Epoch 11/65
38/38 [==============================] - 4s 108ms/step - loss: 0.0043
Epoch 12/65
38/38 [==============================] - 4s 109ms/step - loss: 0.0043
Epoch 13/65
38/38 [=====

In [4]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
# 將訓練集和測試集合併以便進行特徵縮放
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
# 擷取測試集所需用到的數據
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling

X_test = []
for i in range(60, 80):  # timesteps一樣60； 80 = 先前的60天資料+2017年的20天資料
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape 成 3-dimension
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # 將預測結果逆向轉換為原始尺度


1/1 [==============================] - 3s 3s/step
